# pymysql
- mysql을 파이썬에서 사용할 수 있는 라이브러리

```shell
pip install pymysql
```

In [1]:
%pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [325]:
import pymysql
import yaml

In [326]:
DATA_PATH = "data/"

with open(f"{DATA_PATH}config.yml", "r") as f:
    CFG = yaml.full_load(f)

# connect 함수
- MySQL 서버와 연결할 수 있는 함수
- 연결객체를 반환
- 주요 파라미터
    - user: 계정
    - passwd: 패스워드
    - host: 호스트정보(ip 또는 도메인)
    - port: 포트정보
    - db: 선택할 DB 이름

In [327]:
conn = pymysql.connect(**CFG, db="mydata")
conn # 연결객체

# cursor 메서드
- 연결 객체로부터 cursor 객체를 반환
- cursor 객체란?
    - SQL 구문을 실행하기 위한 객체

In [328]:
cursor = conn.cursor()
cursor # 커서객체

## execute 메서드
- 커서객체의 메서드로 전달받은 SQL문을 실행

In [329]:
sql = "SELECT * FROM tb_titanic;"
cursor.execute(sql)

1309

## fetchall 메서드
- 커서객체의 메서드로 execute 메서드 실행 후 쿼리 실행 결과를 모두 가져올 때 사용

In [330]:
result = cursor.fetchall()
result

((1,
  0,
  3,
  'Braund, Mr. Owen Harris',
  'male',
  22.0,
  1,
  0,
  'A/5 21171',
  7.25,
  None,
  'S'),
 (2,
  1,
  1,
  'Cumings, Mrs. John Bradley (Florence Briggs Thayer)',
  'female',
  38.0,
  1,
  0,
  'PC 17599',
  71.2833,
  'C85',
  'C'),
 (3,
  1,
  3,
  'Heikkinen, Miss. Laina',
  'female',
  26.0,
  0,
  0,
  'STON/O2. 3101282',
  7.925,
  None,
  'S'),
 (4,
  1,
  1,
  'Futrelle, Mrs. Jacques Heath (Lily May Peel)',
  'female',
  35.0,
  1,
  0,
  '113803',
  53.1,
  'C123',
  'S'),
 (5,
  0,
  3,
  'Allen, Mr. William Henry',
  'male',
  35.0,
  0,
  0,
  '373450',
  8.05,
  None,
  'S'),
 (6,
  0,
  3,
  'Moran, Mr. James',
  'male',
  None,
  0,
  0,
  '330877',
  8.4583,
  None,
  'Q'),
 (7,
  0,
  1,
  'McCarthy, Mr. Timothy J',
  'male',
  54.0,
  0,
  0,
  '17463',
  51.8625,
  'E46',
  'S'),
 (8,
  0,
  3,
  'Palsson, Master. Gosta Leonard',
  'male',
  2.0,
  3,
  1,
  '349909',
  21.075,
  None,
  'S'),
 (9,
  1,
  3,
  'Johnson, Mrs. Oscar W (Elisabeth Vi

In [331]:
cursor.fetchall() # execute 실행 결과를 한번 반환하면 더 이상 반환하지 않음

()

- dict 형식으로 받아오기

In [333]:
from pymysql.cursors import DictCursor
cursor = conn.cursor(DictCursor)

In [334]:
sql = "SELECT deptno, AVG(sal) FROM emp GROUP BY deptno"

In [335]:
cursor.execute(sql)

3

In [336]:
result = cursor.fetchall()

In [337]:
result[:2]

[{'deptno': 10, 'AVG(sal)': Decimal('2916.666667')},
 {'deptno': 20, 'AVG(sal)': Decimal('2175.000000')}]

In [338]:
cursor.execute(sql)
for row in cursor.fetchall():
    print(row)

{'deptno': 10, 'AVG(sal)': Decimal('2916.666667')}
{'deptno': 20, 'AVG(sal)': Decimal('2175.000000')}
{'deptno': 30, 'AVG(sal)': Decimal('1566.666667')}


In [339]:
sql = "SELECT * FROM tb_titanic;"

cursor.execute(sql)

1309

## fetchone 메서드
- 하나씩 가져오기

In [346]:
cursor.fetchone()

{'passengerid': 7,
 'survived': 0,
 'pclass': 1,
 'name': 'McCarthy, Mr. Timothy J',
 'gender': 'male',
 'age': 54.0,
 'sibsp': 0,
 'parch': 0,
 'ticket': '17463',
 'fare': 51.8625,
 'cabin': 'E46',
 'embarked': 'S'}

## 커서 닫아주기

In [347]:
cursor.close()

In [348]:
# cursor.execute(sql) # 커서가 닫혔기 때문에 에러발생

In [349]:
with conn.cursor(DictCursor) as cursor:
    sql = "SELECT * FROM tb_titanic;"
    cursor.execute(sql)
    result = cursor.fetchall()

In [264]:
# cursor.execute(sql)

# 연결 닫아주기

In [352]:
conn.close()

# DB 선택 없이 접속

In [353]:
conn = pymysql.connect(**CFG) # autocommit=True
conn

- autocommit 확인

In [354]:
conn.autocommit_mode

False

## DB 선택하기

In [355]:
conn.select_db('mydata') # USE mydata

In [356]:
cursor = conn.cursor(DictCursor)

In [357]:
import json
with open(f"{DATA_PATH}state.json", "r") as f:
    data = json.load(f)
type(data)

list

In [358]:
len(data)

50

In [359]:
data[0] # 주, 인구수, 10만명 당 살인률

['Alabama', 4779736, 5.7]

In [360]:
sql = """
CREATE TABLE IF NOT EXISTS tb_murder_rate(
    id INT UNSIGNED AUTO_INCREMENT PRIMARY KEY,
    state VARCHAR(50) UNIQUE NOT NULL,
    population INT,
    rate FLOAT
);
"""
cursor.execute(sql)

0

In [361]:
columns_list = ["state", "population", "rate"]

In [363]:
values = [f"'{item}'" if type(item) is str else str(item) for item in data[0]]
",".join(values)

"'Alabama',4779736,5.7"

In [364]:
sql = f"""
INSERT INTO tb_murder_rate({",".join(columns_list)})
VALUES({",".join(values)});
"""
print(sql)


INSERT INTO tb_murder_rate(state,population,rate)
VALUES('Alabama',4779736,5.7);



In [365]:
cursor.execute(sql)

IntegrityError: (1062, "Duplicate entry 'Alabama' for key 'tb_murder_rate.state'")

In [366]:
conn.rollback() # INSERT, DELETE, UPDATE 을 취소

## PlaceHolder
- 동적 SQL 문을 구성할 때 유용하게 사용할 수 있는 기능
- 동적으로 들어갈 위치에 `%s` 를 이용해 SQL을 만든다.

In [367]:
sql = f"""
    INSERT INTO tb_murder_rate({",".join(columns_list)})
    VALUES(%s, %s, %s)
"""
print(sql)


    INSERT INTO tb_murder_rate(state,population,rate)
    VALUES(%s, %s, %s)



In [316]:
cursor.execute(sql, data[0])

1

In [319]:
conn.rollback()

In [320]:
cursor.executemany(sql, data)

50

In [321]:
conn.commit() # 실제 데이터베이스에 반영

- 조회할 때 사용해보기

In [322]:
lst = [100000, 5]
sql = "SELECT * FROM tb_murder_rate WHERE population > %s and rate > %s"

cursor.execute(sql, lst)

16

In [323]:
result = cursor.fetchall()
result

[{'id': 56, 'state': 'Alabama', 'population': 4779736, 'rate': 5.7},
 {'id': 57, 'state': 'Alaska', 'population': 710231, 'rate': 5.6},
 {'id': 59, 'state': 'Arkansas', 'population': 2915918, 'rate': 5.6},
 {'id': 63, 'state': 'Delaware', 'population': 897934, 'rate': 5.8},
 {'id': 64, 'state': 'Florida', 'population': 18801310, 'rate': 5.8},
 {'id': 65, 'state': 'Georgia', 'population': 9687653, 'rate': 5.7},
 {'id': 68, 'state': 'Illinois', 'population': 12830632, 'rate': 5.3},
 {'id': 73, 'state': 'Louisiana', 'population': 4533372, 'rate': 10.3},
 {'id': 75, 'state': 'Maryland', 'population': 5773552, 'rate': 6.1},
 {'id': 77, 'state': 'Michigan', 'population': 9883640, 'rate': 5.4},
 {'id': 79, 'state': 'Mississippi', 'population': 2967297, 'rate': 8.6},
 {'id': 80, 'state': 'Missouri', 'population': 5988927, 'rate': 6.6},
 {'id': 83, 'state': 'Nevada', 'population': 2700551, 'rate': 6.0},
 {'id': 88, 'state': 'North Carolina', 'population': 9535483, 'rate': 5.1},
 {'id': 95, 'sta

In [324]:
cursor.close()
conn.close()